In [4]:
import cv2
import mediapipe as mp
import threading
import numpy as np

In [ ]:

# Inicializar MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)
pose_video2 = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)

# Carregar vídeos
video_camera = cv2.VideoCapture(0)
video_file = cv2.VideoCapture('media/videoplayback.mp4')

# Inicializar frames
frame_cam = None
frame_file = None
output_cam = None
output_file = None

# Função para processar a pose
def detectPose(image, pose):
    output_image = np.zeros_like(image)
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(imageRGB)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS)
    return output_image

# Função para capturar e processar o frame da câmera
def process_camera():
    global frame_cam, output_cam
    while video_camera.isOpened():
        ok, frame = video_camera.read()
        if not ok:
            break
        frame = cv2.flip(frame, 1)
        frame = cv2.resize(frame, (640, 480))
        frame_cam = frame
        output_cam = detectPose(frame, pose_video)

# Função para capturar e processar o frame do vídeo
def process_video():
    global frame_file, output_file
    while video_file.isOpened():
        ok2, frame2 = video_file.read()
        if not ok2:
            break
        frame2 = cv2.resize(frame2, (640, 480))
        frame_file = frame2
        output_file = detectPose(frame2, pose_video2)

# Iniciar threads
thread_cam = threading.Thread(target=process_camera)
thread_video = threading.Thread(target=process_video)

thread_cam.start()
thread_video.start()

# Loop para exibir as janelas
cv2.namedWindow('Video Original', cv2.WINDOW_NORMAL)
cv2.namedWindow('Meu Esqueleto', cv2.WINDOW_NORMAL)
cv2.namedWindow('Esqueleto do Video', cv2.WINDOW_NORMAL)

while True:
    # Mostrar os frames processados
    if frame_cam is not None and output_cam is not None:
        cv2.imshow('Meu Esqueleto', output_cam)
    if frame_file is not None and output_file is not None:
        cv2.imshow('Esqueleto do Video', output_file)       
        cv2.imshow('Video Original', frame_file)




    # Pressionar 'q' para sair
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
video_camera.release()
video_file.release()
cv2.destroyAllWindows()

# Esperar as threads terminarem
thread_cam.join()
thread_video.join()
